# importation des bibliotheque

In [95]:
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import math
import keras
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.preprocessing import LabelEncoder
style.use('fivethirtyeight')
%matplotlib inline
%pylab inline



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['max', 'rand']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


# importation des données

In [96]:
train = pd.read_csv(
    "/content/training_data.csv",
    delimiter=';',  # Spécifiez le délimiteur
    on_bad_lines='skip',  # Ignore les lignes problématiques
    engine='python'       # Utilisez le moteur Python pour plus de tolérance
)

train.head()

,V1,V2,V3,V4,V5,V6,V7,V8,PDC,Contrat,localite,MeterMatricule,Millesime,T0,Label
0,152.0,0.99,0.897,1.67,5,54,37743,3,11,LIANCOURTOIS,LIANCOURTOIS,NaN,C17FA,08/12/2017,STOPPED
1,76.0,1.00,0.993,0.00,0,239,413425,1,1701,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,29/10/2016,STOPPED
2,71.0,0.32,0.965,8.38,53,45,85193,8,1802,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,28/11/2016,STOPPED
3,92.0,0.84,0.871,4.61,77,49,263697,36,2201,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,29/10/2017,STOPPED
4,91.0,0.31,0.189,28.95,126,13,90206,27,5201,NEUVY-LES-MOULINS,NEUVY,NaN,C12FA,19/08/2018,STOPPED


In [97]:
data=train.copy()

In [98]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64826 entries, 0 to 64825
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   V1              64826 non-null  float64
 1   V2              64826 non-null  float64
 2   V3              64826 non-null  float64
 3   V4              64826 non-null  float64
 4   V5              64826 non-null  int64  
 5   V6              64826 non-null  int64  
 6   V7              64826 non-null  int64  
 7   V8              64826 non-null  int64  
 8   PDC             64826 non-null  object 
 9   Contrat         64826 non-null  object 
 10  localite        64826 non-null  object 
 11  MeterMatricule  0 non-null      float64
 12  Millesime       64825 non-null  object 
 13  T0              64825 non-null  object 
 14  Label           64825 non-null  object 
dtypes: float64(5), int64(4), object(6)
memory usage: 7.4+ MB


In [99]:
data.shape

(64826, 15)

In [100]:
data.isnull().sum()

,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
PDC,0
Contrat,0


# Netoillage des variables
Nous retiendrons que les variables interet

In [101]:
data.head(20)

,V1,V2,V3,V4,V5,V6,V7,V8,PDC,Contrat,localite,MeterMatricule,Millesime,T0,Label
0,152.0,0.99,0.897,1.67,5,54,37743,3,11,LIANCOURTOIS,LIANCOURTOIS,NaN,C17FA,08/12/2017,STOPPED
1,76.0,1.00,0.993,0.00,0,239,413425,1,1701,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,29/10/2016,STOPPED
2,71.0,0.32,0.965,8.38,53,45,85193,8,1802,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,28/11/2016,STOPPED
3,92.0,0.84,0.871,4.61,77,49,263697,36,2201,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,29/10/2017,STOPPED
4,91.0,0.31,0.189,28.95,126,13,90206,27,5201,NEUVY-LES-MOULINS,NEUVY,NaN,C12FA,19/08/2018,STOPPED
5,65.0,0.74,0.451,12.57,60,33,187459,7,5901,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,30/09/2016,STOPPED
6,63.0,0.08,0.185,43.50,166,21,185199,8,6801,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,01/05/2017,STOPPED
7,62.0,0.12,0.049,215.00,426,2,5338,2,108,LIANCOURTOIS,LIANCOURTOIS,NaN,C17FA,11/09/2018,STOPPED
8,122.0,0.99,0.635,38.00,114,191,113709,3,113,LIANCOURTOIS,LIANCOURTOIS,NaN,C17FA,03/08/2018,STOPPED
9,238.0,0.78,0.943,5.80,19,149,103712,5,144,LIANCOURTOIS,LIANCOURTOIS,NaN,C17FA,21/03/2018,STOPPED


In [102]:
data.isna().sum()

,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
PDC,0
Contrat,0


In [103]:
data.isnull().sum()

,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
PDC,0
Contrat,0


# Question 1: Calculer la variable v9



In [104]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,PDC,Contrat,localite,MeterMatricule,Millesime,T0,Label
0,152.0,0.99,0.897,1.67,5,54,37743,3,11,LIANCOURTOIS,LIANCOURTOIS,NaN,C17FA,08/12/2017,STOPPED
1,76.0,1.00,0.993,0.00,0,239,413425,1,1701,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,29/10/2016,STOPPED
2,71.0,0.32,0.965,8.38,53,45,85193,8,1802,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,28/11/2016,STOPPED
3,92.0,0.84,0.871,4.61,77,49,263697,36,2201,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,29/10/2017,STOPPED
4,91.0,0.31,0.189,28.95,126,13,90206,27,5201,NEUVY-LES-MOULINS,NEUVY,NaN,C12FA,19/08/2018,STOPPED


In [105]:
data["Millesime"].unique()

array(['C17FA', 'C13FA', 'C12FA', 'C16FA', 'C18VA', 'C17VA', 'C17SB',
       'C17SV', 'C16VA', 'C17LA', 'C18SB', 'C18SV', 'C16SB', 'C12SE',
       'C14FA', 'C11FA', 'C15SV', 'C10FA', 'C15FA', 'C12SB', 'C13SD',
       'C12LA', 'C11SB', 'C15JG', 'C11SA', 'C18FA', 'C13SE', 'C10EB',
       'C10SB', 'C13LA', 'C12SU', 'C16SV', '27246', '68774', 'C11SE',
       'C17SE', 'C17SU', 'D12DA', 'D11KA', 'D12JA', 'D12KA', 'C12SD',
       'C11UA', 'C12SA', 'D11MA', 'C13SA', 'I12JA', 'C14LA', 'C15UA',
       'C15LA', 'C16LA', 'C12SV', 'C13SV', 'C14JI', 'C08KH', 'C12UA',
       'C13JG', 'C11JG', 'C10EE', 'C14JG', 'C13KI', 'C13SB', 'D10KA',
       'C10SA', 'D12LA', 'C14SA', 'C10EA', 'C13UA', 'C14UA', 'C10UA',
       'D10MA', 'C11LA', 'N11JA', 'N12JA', 'C15SA', 'C16SA', 'C11SU',
       'C14SD', 'D06HA', 'D06DA', 'D08MA', 'D11DA', 'D05DA', 'D05HA',
       'D07MA', 'D08LA', 'D08DA', 'D13MA', 'D09MA', 'D13DA', 'C10LA',
       'D11JA', 'D10JA', 'N11PE', 'D02PA', 'I11JA', 'C09EB', 'D10LA',
       'C10PA', 'D14

In [106]:


def calculate_v9(df):

# Extraction des 5 premiers caractères de la colonne 'sensor' pour obtenir le 'millesime'
    df['Millesime'] = df['Millesime'].str[:5]

# Conditions pour définir v9
    condition1 = (df['Millesime'].isin(['C10FA', 'C10LA', 'C10SA', 'C11FA', 'C11LA', 'C11SA'])) & (df['V7'].astype(str).str.len() > 3)
    condition2 = df['Millesime'].isin(['D16BU', 'Z12ER', 'C07AA'])

# Calcul de v9 selon les règles
    df['v9'] = 2  # Par défaut
    df.loc[condition1, 'v9'] = 0
    df.loc[condition2, 'v9'] = 1

    return df['v9']





In [107]:
calculate_v9(data)

,v9
0,2
1,2
2,2
3,2
4,2
...,...
64821,2
64822,2
64823,2
64824,2


In [108]:
data.head(40)

,V1,V2,V3,V4,V5,V6,V7,V8,PDC,Contrat,localite,MeterMatricule,Millesime,T0,Label,v9
0,152.0,0.99,0.897,1.67,5,54,37743,3,11,LIANCOURTOIS,LIANCOURTOIS,NaN,C17FA,08/12/2017,STOPPED,2
1,76.0,1.00,0.993,0.00,0,239,413425,1,1701,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,29/10/2016,STOPPED,2
2,71.0,0.32,0.965,8.38,53,45,85193,8,1802,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,28/11/2016,STOPPED,2
3,92.0,0.84,0.871,4.61,77,49,263697,36,2201,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,29/10/2017,STOPPED,2
4,91.0,0.31,0.189,28.95,126,13,90206,27,5201,NEUVY-LES-MOULINS,NEUVY,NaN,C12FA,19/08/2018,STOPPED,2
5,65.0,0.74,0.451,12.57,60,33,187459,7,5901,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,30/09/2016,STOPPED,2
6,63.0,0.08,0.185,43.50,166,21,185199,8,6801,NEUVY-LES-MOULINS,NEUVY,NaN,C13FA,01/05/2017,STOPPED,2
7,62.0,0.12,0.049,215.00,426,2,5338,2,108,LIANCOURTOIS,LIANCOURTOIS,NaN,C17FA,11/09/2018,STOPPED,2
8,122.0,0.99,0.635,38.00,114,191,113709,3,113,LIANCOURTOIS,LIANCOURTOIS,NaN,C17FA,03/08/2018,STOPPED,2
9,238.0,0.78,0.943,5.80,19,149,103712,5,144,LIANCOURTOIS,LIANCOURTOIS,NaN,C17FA,21/03/2018,STOPPED,2


In [109]:
data1=data.copy()

In [110]:
data1.drop(["Millesime"],inplace=True,axis=1)

In [111]:
data1.head(40)

,V1,V2,V3,V4,V5,V6,V7,V8,PDC,Contrat,localite,MeterMatricule,T0,Label,v9
0,152.0,0.99,0.897,1.67,5,54,37743,3,11,LIANCOURTOIS,LIANCOURTOIS,NaN,08/12/2017,STOPPED,2
1,76.0,1.00,0.993,0.00,0,239,413425,1,1701,NEUVY-LES-MOULINS,NEUVY,NaN,29/10/2016,STOPPED,2
2,71.0,0.32,0.965,8.38,53,45,85193,8,1802,NEUVY-LES-MOULINS,NEUVY,NaN,28/11/2016,STOPPED,2
3,92.0,0.84,0.871,4.61,77,49,263697,36,2201,NEUVY-LES-MOULINS,NEUVY,NaN,29/10/2017,STOPPED,2
4,91.0,0.31,0.189,28.95,126,13,90206,27,5201,NEUVY-LES-MOULINS,NEUVY,NaN,19/08/2018,STOPPED,2
5,65.0,0.74,0.451,12.57,60,33,187459,7,5901,NEUVY-LES-MOULINS,NEUVY,NaN,30/09/2016,STOPPED,2
6,63.0,0.08,0.185,43.50,166,21,185199,8,6801,NEUVY-LES-MOULINS,NEUVY,NaN,01/05/2017,STOPPED,2
7,62.0,0.12,0.049,215.00,426,2,5338,2,108,LIANCOURTOIS,LIANCOURTOIS,NaN,11/09/2018,STOPPED,2
8,122.0,0.99,0.635,38.00,114,191,113709,3,113,LIANCOURTOIS,LIANCOURTOIS,NaN,03/08/2018,STOPPED,2
9,238.0,0.78,0.943,5.80,19,149,103712,5,144,LIANCOURTOIS,LIANCOURTOIS,NaN,21/03/2018,STOPPED,2


# Question2 :. On souhaite prédire si un compteur est bloqué ou stoppé en fonction des différentes variables v1 à v9.



# Question c :Quelle est la proportion de compteur bloqué/stoppé par typologie de localite

In [112]:
# Comptage total par typologie de localité
total_par_localite = data.groupby('localite')['Label'].count()

# Comptage des compteurs bloqués/stoppés par typologie de localité
etat_par_localite = data.groupby(['localite', 'Label']).size().unstack(fill_value=0)

# Calcul des proportions
proportions = etat_par_localite.div(total_par_localite, axis=0)

print(proportions)

Label          BLOCKED   STOPPED
localite                        
AGUR-LCF      0.000000  1.000000
APT           0.000000  1.000000
AREZZO        0.000000  1.000000
CALEDONIE     0.000000  1.000000
CHAMPFLEUR    0.005848  0.994152
CREMA         0.000000  1.000000
FLORENCE      0.000000  1.000000
LAFLECHE      0.000000  1.000000
LEMANSMETRO   0.000000  1.000000
LIANCOURTOIS  0.000000  1.000000
LYO                NaN       NaN
LYONNAISE     0.011156  0.988844
MARTINIQUE    0.000000  1.000000
MONTDEMARSAN  0.000000  1.000000
NEUVY         0.000000  1.000000
NICEREA       0.000000  1.000000
OCEAVHF       0.004639  0.995361
OMFLEURY      0.000000  1.000000
POLYNESIE     0.000000  1.000000
SIDERM        0.002817  0.997183
SIEVA         0.000000  1.000000
VALBONNE      0.000000  1.000000


#Question D: Modéliser à l’aide d’un modèle de machine learning la prediction de compteur bloqué


In [113]:
data1["Label"].unique()

array(['STOPPED', 'BLOCKED', None], dtype=object)

In [114]:
X=data1[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8','v9']]
Y=data1['Label']
print(X.shape)
print(Y.shape)

(64826, 9)
(64826,)


In [115]:
le = LabelEncoder()
Y= le.fit_transform(Y)

In [116]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=50)


In [117]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(51860, 9)
(12966, 9)
(51860,)
(12966,)


# randomForest

In [118]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

In [119]:

model = RandomForestClassifier(random_state=50)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=50)

In [120]:
y_pred = model.predict(X_test)


In [121]:

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.96      0.28      0.44        85
           1       1.00      1.00      1.00     12881

    accuracy                           1.00     12966
   macro avg       0.98      0.64      0.72     12966
weighted avg       1.00      1.00      0.99     12966



In [122]:
"""# la robustesse du modele
scores = cross_val_score(model, X, Y, cv=5)
print("Cross-Validation Scores:", scores)"""


'# la robustesse du modele\nscores = cross_val_score(model, X, Y, cv=5)\nprint("Cross-Validation Scores:", scores)'

In [123]:
"""from sklearn.metrics import roc_auc_score
y_pred_prob = model.predict_proba(X_test)[:, 1]
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_prob))"""


'from sklearn.metrics import roc_auc_score\ny_pred_prob = model.predict_proba(X_test)[:, 1]\nprint("ROC-AUC Score:", roc_auc_score(y_test, y_pred_prob))'

# BAGGING


In [124]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [125]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Configuration du modèle de base (arbre de décision)
base_model = DecisionTreeClassifier(random_state=42)

# Configuration du BaggingClassifier
bagging_model = BaggingClassifier(
    base_model,               # Modèle de base
    n_estimators=50,          # Nombre d'échantillons bootstrap
    max_samples=0.8,          # Fraction de l'ensemble de données utilisé pour chaque bootstrap
    random_state=50,
    n_jobs=-1                 # Utilisation de tous les cœurs disponibles
)

# Entraînement du modèle
bagging_model.fit(X_train, y_train)




BaggingClassifier(estimator=DecisionTreeClassifier(random_state=42),
                  max_samples=0.8, n_estimators=50, n_jobs=-1, random_state=50)

In [126]:
# Prédictions
y_pred = bagging_model.predict(X_test)

# Évaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9951411383618695

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.29      0.44        85
           1       1.00      1.00      1.00     12881

    accuracy                           1.00     12966
   macro avg       0.94      0.65      0.72     12966
weighted avg       0.99      1.00      0.99     12966



### Nos constatons que le Bagging est le meilleur modele la retenons donc
Ici Nous utilisons uniquement des modele ensembliste car offrand des meilleurs resultat  

### Nous enreistrons le modele pour pourvoir l'utiliser plus tard

In [127]:
import joblib

# Enregistrer le modèle entraîné dans un fichier
joblib.dump(bagging_model, 'bagging_model.pkl')


['bagging_model.pkl']

# PARTIE 2

In [128]:
dataset_sample_spark = pd.read_csv(
    "/content/dataset_sample_spark.csv",
    delimiter=';',  # Spécifiez le délimiteur
    on_bad_lines='skip',  # Ignore les lignes problématiques
    engine='python'       # Utilisez le moteur Python pour plus de tolérance
)



In [129]:
dataset_sample_spark.sample(20)

,sensor,servicePoint,client,transmitter,date,meterDiameter,volume_l_value,index_interpolated_d_l_value
638,C04AE134021,"9,84588E+11",LYONNAISE,C01E00D9,30/09/2020,40,6414.0,6.634548e+06
460,C04AE134021,"9,84588E+11",LYONNAISE,C01E00D9,05/04/2020,40,28.0,6.061770e+06
4378,C09FA030008,"9,82018E+11",LYONNAISE,C241E7A5,07/08/2019,15,400.0,1.823715e+06
6799,C10ED005356,"9,86814E+11",LYONNAISE,C2AA2108,23/01/2019,30,1107.0,1.573456e+06
3248,C08FA041975,"9,80564E+11",LYONNAISE,C2412C78,16/01/2021,15,0.0,6.004790e+05
1584,C07AA092853,"9,89432E+11",LYONNAISE,11A000060C8,25/09/2020,15,744.0,3.487900e+06
3986,C08FA085851,"9,84967E+11",LYONNAISE,C2ABDA49,11/10/2020,15,153.0,1.189360e+06
1898,C08EB001382,"9,85226E+11",LYONNAISE,C24F74E5,29/06/2019,20,0.0,4.875640e+05
7774,C10FA046664,"9,85156E+11",LYONNAISE,11A5190040261007,12/08/2019,15,0.0,3.558200e+05
3952,C08FA085851,"9,84967E+11",LYONNAISE,C2ABDA49,07/09/2020,15,399.0,1.181393e+06


In [130]:
dataset_sample_spark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8438 entries, 0 to 8437
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   sensor                        8438 non-null   object 
 1   servicePoint                  8438 non-null   object 
 2   client                        8438 non-null   object 
 3   transmitter                   8438 non-null   object 
 4   date                          8438 non-null   object 
 5   meterDiameter                 8438 non-null   int64  
 6   volume_l_value                8421 non-null   float64
 7   index_interpolated_d_l_value  8417 non-null   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 527.5+ KB


## QUESTION a:
Rassurez-vous que la triplette (sensor, servicePoint, transmitter) qui définit un
compteur est unique pour une date donnée. Si ce n’était pas le cas, supprimez
les doublons.


In [131]:
# Convertir la colonne 'date' avec le bon format (jour/mois/année)
dataset_sample_spark['date'] = pd.to_datetime(dataset_sample_spark['date'], dayfirst=True, errors='coerce')


In [132]:
dataset_sample_spark[['sensor', 'servicePoint', 'transmitter', 'date']].duplicated().sum()


0

### On constate qu'il n'y a pas de valeur dupliquée chaque ligne du dataframe est donc unique.cela signifie chaque ligne du jeu de donnée represente la consommation d'un compteur a une date donnée.

# Question b
 Quelle est la consommation moyenne par diametre de compteur ? En déduire quels sont les gros compteurs (compteurs qui consomment le plus).


In [133]:
dataset_sample_spark.head(15)

,sensor,servicePoint,client,transmitter,date,meterDiameter,volume_l_value,index_interpolated_d_l_value
0,C04AE134021,"9,84588E+11",LYONNAISE,C01E00D9,2019-01-01,40,0.0,5623306.0
1,C04AE134021,"9,84588E+11",LYONNAISE,C01E00D9,2019-01-02,40,61.0,5623306.0
2,C04AE134021,"9,84588E+11",LYONNAISE,C01E00D9,2019-01-03,40,7.0,5623367.0
3,C04AE134021,"9,84588E+11",LYONNAISE,C01E00D9,2019-01-04,40,5.0,5623374.0
4,C04AE134021,"9,84588E+11",LYONNAISE,C01E00D9,2019-01-05,40,3.0,5623379.0
5,C04AE134021,"9,84588E+11",LYONNAISE,C01E00D9,2019-01-06,40,1200.0,5623382.0
6,C04AE134021,"9,84588E+11",LYONNAISE,C01E00D9,2019-01-07,40,1076.0,5624582.0
7,C04AE134021,"9,84588E+11",LYONNAISE,C01E00D9,2019-01-08,40,2346.0,5625658.0
8,C04AE134021,"9,84588E+11",LYONNAISE,C01E00D9,2019-01-09,40,4533.0,5628004.0
9,C04AE134021,"9,84588E+11",LYONNAISE,C01E00D9,2019-01-10,40,1165.0,5632537.0


In [134]:
average_consumption = dataset_sample_spark.groupby('meterDiameter').agg( consommation_moyenne=('volume_l_value', 'mean') )
average_consumption

,consommation_moyenne
meterDiameter,
15,409.397057
20,99.495732
30,629.811105
40,1529.341151


on constate que la categorie de compteur qui consommme le plus est ceux ayant un diametre egale a 15 avec une consommation moyenne de 409.397057

#  Calculer les variables v1 à v9 comme décrites ci-dessous


In [135]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

# 1. Configurer Spark
spark = SparkSession.builder \
    .appName("ClassificationModel") \
    .getOrCreate()

In [136]:
import pyspark.sql.functions as F

# Remplacer ';' par le séparateur réel utilisé dans votre fichier CSV
data = spark.read.csv('/content/dataset_sample_spark.csv',
                      header=True,
                      inferSchema=True,
                      sep=';')  # Spécifie le point-virgule comme séparateur

In [137]:
# Afficher les premières lignes pour vérifier les données
data.show()


+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+
|     sensor|servicePoint|   client|transmitter|      date|meterDiameter|volume_l_value|index_interpolated_d_l_value|
+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|01/01/2019|           40|           0.0|                   5623306.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|02/01/2019|           40|          61.0|                   5623306.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|03/01/2019|           40|           7.0|                   5623367.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|04/01/2019|           40|           5.0|                   5623374.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|05/01/2019|           40|           3.0|                   5623379.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|06/01/20

In [138]:
# Nombre de lignes
num_rows = data.count()

# Nombre de colonnes
num_columns = len(data.columns)

print(f"Le DataFrame a {num_rows} lignes et {num_columns} colonnes.")


Le DataFrame a 8438 lignes et 8 colonnes.


## Variable v1 : Temps écoulé sans consommation.


In [139]:

from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F



### Nous commencons par suprimer toutes lignes donc la consommation est inferieur a 20

In [140]:
# Supprimer les lignes avec consommation non significative (volume_l_value <= 20)
data = data.filter(F.col("volume_l_value") > 20)

# Afficher les résultats pour validation
data.show()


+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+
|     sensor|servicePoint|   client|transmitter|      date|meterDiameter|volume_l_value|index_interpolated_d_l_value|
+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|02/01/2019|           40|          61.0|                   5623306.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|06/01/2019|           40|        1200.0|                   5623382.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|07/01/2019|           40|        1076.0|                   5624582.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|08/01/2019|           40|        2346.0|                   5625658.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|09/01/2019|           40|        4533.0|                   5628004.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|10/01/20

In [141]:
# Initialiser SparkSession
spark = SparkSession.builder.appName("DerniereDateFonctionnement").getOrCreate()

## Nous creer un colonne "compteur" qui est la concactenation des colonne sensor,servicePoint,transmitter. nous groupons ensuite les dates par compteur.

In [142]:
# Créer une colonne qui identifie chaque compteur
data = data.withColumn("compteur", F.concat_ws("_", F.col("sensor"), F.col("servicePoint"), F.col("transmitter")))
# Définir une fenêtre partitionnée par compteur, triée par date
window_spec = Window.partitionBy("compteur").orderBy("date")

In [143]:
data.show(40)

+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+--------------------+
|     sensor|servicePoint|   client|transmitter|      date|meterDiameter|volume_l_value|index_interpolated_d_l_value|            compteur|
+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+--------------------+
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|02/01/2019|           40|          61.0|                   5623306.0|C04AE134021_9,845...|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|06/01/2019|           40|        1200.0|                   5623382.0|C04AE134021_9,845...|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|07/01/2019|           40|        1076.0|                   5624582.0|C04AE134021_9,845...|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|08/01/2019|           40|        2346.0|                   5625658.0|C04AE134021_9,845...|
|C04AE134021| 9,84588E+11|L

In [144]:


# Définir une fenêtre partitionnée par compteur
window_spec = Window.partitionBy("compteur")

# Ajouter une colonne pour la date maximale par compteur
data = data.withColumn("T0", F.max("date").over(window_spec))

# Filtrer pour garder uniquement les lignes où la date est la date maximale
data_max_date = data.filter(F.col("date") == F.col("T0"))

# Sélectionner les colonnes pertinentes
data_max_date.show()




+-----------+------------+---------+----------------+----------+-------------+--------------+----------------------------+--------------------+----------+
|     sensor|servicePoint|   client|     transmitter|      date|meterDiameter|volume_l_value|index_interpolated_d_l_value|            compteur|        T0|
+-----------+------------+---------+----------------+----------+-------------+--------------+----------------------------+--------------------+----------+
|C04AE134021| 9,84588E+11|LYONNAISE|        C01E00D9|31/12/2019|           40|         101.0|                   5991315.0|C04AE134021_9,845...|31/12/2019|
|C07AA092853| 9,89432E+11|LYONNAISE|     11A000060C8|31/12/2019|           15|         534.0|                   3360343.0|C07AA092853_9,894...|31/12/2019|
|C08EB001382| 9,85226E+11|LYONNAISE|        C24F74E5|31/10/2020|           20|         174.0|                    542339.0|C08EB001382_9,852...|31/10/2020|
|C08FA041975| 9,80564E+11|LYONNAISE|        C2412C78|31/12/2019|      

## Nous identifions ici la date maximal consideree comme date d'analyse.

In [145]:
from pyspark.sql.functions import max


# Nous identifions ici la date maximale parmis les dernières date de consommation des compteurs
# Supposons que "data" est votre DataFrame et "column_name" est la colonne où vous souhaitez trouver le maximum.
# Nous utiliserons cette date comme date de l'analyse "T"


max_value = data_max_date.select(max("T0")).collect()[0][0]
print("La valeur maximale de la colonne est :", max_value)

La valeur maximale de la colonne est : 31/12/2020


### Nous creons la variable v1 qui identifie le nombre de jours separant le dernier jour a laquelle le comteur a cesser de fonctionner et la date d'analyse

In [146]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, to_date

# Définir explicitement T comme une date fixe (2020-12-31)
# nous utilisons cette methode car elle est flexible et permet de modifier la date
data_max_date = data_max_date.withColumn("T", F.lit("2020-12-31").cast("date"))

#Convertir la colonne To en type date
data_max_date = data_max_date.withColumn("T0", F.to_date(F.col("T0"), "dd/MM/yyyy"))

# Ajouter une colonne "v1" en utilisant T
data_max_date = data_max_date.withColumn(
    "v1",
    F.when(
        F.col("T0").isNotNull(),
        F.datediff(F.col("T"), F.col("T0"))  # T - To
    ).otherwise(730)  # Cas d'exclusion : convention de 2 ans
)


In [147]:
dataV1=data_max_date.select("compteur",     "T0"   , "v1")
dataV1.show()

+--------------------+----------+---+
|            compteur|        T0| v1|
+--------------------+----------+---+
|C04AE134021_9,845...|2019-12-31|366|
|C07AA092853_9,894...|2019-12-31|366|
|C08EB001382_9,852...|2020-10-31| 61|
|C08FA041975_9,805...|2019-12-31|366|
|C08FA085851_9,849...|2020-12-31|  0|
|C09FA030008_9,820...|2020-12-31|  0|
|C09FA079673_9,881...|2019-12-31|366|
|C09FA127701_9,856...|2019-10-31|427|
|C10ED005356_9,868...|2020-12-31|  0|
|C10FA046664_9,851...|2019-12-31|366|
+--------------------+----------+---+



# Variable v2 : Taux de consommation non nul sur les 90 jours précédant le To
décrit le pourcentage de jours avec une consommation sur les 90 jours avant To. Si certaines
sont manquantes, le taux est calculé sur les consommations disponibles. Il se peut qu'il n'y ait
pas de données avant To.


In [148]:
#reimportation des variable
data = spark.read.csv('/content/dataset_sample_spark.csv',
                      header=True,
                      inferSchema=True,
                      sep=';')  # Spécifie le point-virgule comme séparateur

In [149]:

# Créer une colonne qui identifie chaque compteur
data = data.withColumn("compteur", F.concat_ws("_", F.col("sensor"), F.col("servicePoint"), F.col("transmitter")))

# Définir une fenêtre ordonnée par date par ordre decroissante pour chaque compteur
window_spec = Window.partitionBy("compteur").orderBy(F.desc("date"))


# Numéroter les dates pour chaque compteur
data = data.withColumn("row_num", F.row_number().over(window_spec))

# Filtrer les 9 premières dates suivant la premiere date qui est la date
# maximale c'est a dire "T0"
data_91 = data.filter((F.col("row_num") > 1) & (F.col("row_num") <= 91))



# Calculer la somme des consommations pour les 90 premières dates suivant la
# premiere date qui est la date maximale
data_91_sum = data_91.groupBy("compteur").agg(
    F.sum("volume_l_value").alias("sum_91_days")
)

# Calculer la consommation totale pour chaque compteur
data_total_sum = data.groupBy("compteur").agg(
    F.sum("volume_l_value").alias("total_sum")
)

# Joindre les deux DataFrames pour calculer le pourcentage
dataV2 = data_91_sum.join(data_total_sum, on="compteur", how="inner")

# Calculer le pourcentage
dataV2 = dataV2.withColumn(
    "V2",
    (F.col("sum_91_days") / F.col("total_sum"))
)

# Afficher le résultat final
dataV2.select("compteur", "V2").show()


+--------------------+-------------------+
|            compteur|                 V2|
+--------------------+-------------------+
|C10ED005356_9,868...|0.07939149607793977|
|C08FA085851_9,849...|0.11176456142361486|
|C07AA092853_9,894...|0.20242521917372824|
|C08EB001382_9,852...|0.08178320241738442|
|C04AE134021_9,845...|0.07793212482141205|
|C08FA041975_9,805...|0.12098275646939412|
|C09FA030008_9,820...|0.09753842843436737|
|C09FA127701_9,856...|0.07410879573207052|
|C10FA046664_9,851...|0.09999954139882081|
|C09FA079673_9,881...|0.10333489458946546|
+--------------------+-------------------+



# Variable v3 : Taux de consommation non nul sur 6 mois (183 jours) avant le To - 90 jours

In [150]:
#reimportation des variable
data = spark.read.csv('/content/dataset_sample_spark.csv',
                      header=True,
                      inferSchema=True,
                      sep=';')  # Spécifie le point-virgule comme séparateur

In [151]:

# Créer une colonne qui identifie chaque compteur
data = data.withColumn("compteur", F.concat_ws("_", F.col("sensor"), F.col("servicePoint"), F.col("transmitter")))

# Définir une fenêtre ordonnée par date par ordre decroissante pour chaque compteur
window_spec = Window.partitionBy("compteur").orderBy(F.desc("date"))


# Numéroter les dates pour chaque compteur
data = data.withColumn("row_num", F.row_number().over(window_spec))

# Filtrer les 183 premières dates suivant la 91 eme qui est la date
# maximale c'est a dire "T0-90" et "T0-273"
data_183 = data.filter((F.col("row_num") > 90) & (F.col("row_num") <= 273))


# Calculer la somme des consommations pour les 90 premières dates suivant la
# premiere date qui est la date maximale
data_183_sum = data_183.groupBy("compteur").agg(
    F.sum("volume_l_value").alias("sum_183_days")
)

# Calculer la consommation totale pour chaque compteur
data_total_sum = data.groupBy("compteur").agg(
    F.sum("volume_l_value").alias("total_sum")
)

# Joindre les deux DataFrames pour calculer le pourcentage
dataV3 = data_183_sum.join(data_total_sum, on="compteur", how="inner")

# Calculer le pourcentage
dataV3 = dataV3.withColumn(
    "V3",
    (F.col("sum_183_days") / F.col("total_sum"))
)

# Afficher le résultat final
dataV3.select("compteur", "V3").show()

+--------------------+-------------------+
|            compteur|                 V3|
+--------------------+-------------------+
|C04AE134021_9,845...|  0.198451011928361|
|C07AA092853_9,894...| 0.2009644806973564|
|C08EB001382_9,852...|0.23286648006513086|
|C08FA041975_9,805...|0.21015767843581087|
|C08FA085851_9,849...|0.22177504981590856|
|C09FA030008_9,820...|0.18731148740529122|
|C09FA079673_9,881...| 0.2013353962259381|
|C09FA127701_9,856...| 0.1947049610443807|
|C10ED005356_9,868...|0.21516000254116543|
|C10FA046664_9,851...|0.20737114392046646|
+--------------------+-------------------+



# Variable v4 : durée moyenne des périodes de consommation nulle.


In [152]:
#reimportation des variable
data = spark.read.csv('/content/dataset_sample_spark.csv',
                      header=True,
                      inferSchema=True,
                      sep=';')  # Spécifie le point-virgule comme séparateur

In [153]:
data.show(60)

+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+
|     sensor|servicePoint|   client|transmitter|      date|meterDiameter|volume_l_value|index_interpolated_d_l_value|
+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|01/01/2019|           40|           0.0|                   5623306.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|02/01/2019|           40|          61.0|                   5623306.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|03/01/2019|           40|           7.0|                   5623367.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|04/01/2019|           40|           5.0|                   5623374.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|05/01/2019|           40|           3.0|                   5623379.0|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|06/01/20

In [154]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Initialiser SparkSession
spark = SparkSession.builder.appName("AnalyseConsommation").getOrCreate()



# Étape 1 : Créer une colonne qui identifie chaque compteur
data = data.withColumn("compteur", F.concat_ws("_", F.col("sensor"), F.col("servicePoint"), F.col("transmitter")))

data = data.withColumn("date", F.to_date("date", "dd/MM/yyyy"))

# Étape 2 : Déterminer T0 (la dernière consommation) pour chaque compteur
window_spec_desc = Window.partitionBy("compteur").orderBy(F.desc("date"))
data = data.withColumn("T0", F.max("date").over(window_spec_desc))


# Étape 3 : Filtrer les données pour les dates jusqu'à T0
data_filtered = data.filter(F.col("date") <= F.col("T0"))
data_filtered.show()

+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+--------------------+----------+
|     sensor|servicePoint|   client|transmitter|      date|meterDiameter|volume_l_value|index_interpolated_d_l_value|            compteur|        T0|
+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+--------------------+----------+
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|2021-08-08|           40|           0.0|                   7070442.0|C04AE134021_9,845...|2021-08-08|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|2021-08-07|           40|           0.0|                   7070442.0|C04AE134021_9,845...|2021-08-08|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|2021-08-06|           40|           0.0|                   7070442.0|C04AE134021_9,845...|2021-08-08|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|2021-08-05|           40|           0.0|            

In [155]:

# Étape 4 : Identifier les consommations nulles (exactement 0)
data_filtered = data_filtered.withColumn(
    "is_null_consumption",
    F.when((F.col("volume_l_value") == 0) & (F.col("volume_l_value") >= 0), 1).otherwise(0)
)

# Étape 5 : Identifier les séries de consommation nulle jusqu'à T0
# Ajouter un identifiant de groupe pour chaque série de consommation nulle
data_filtered = data_filtered.withColumn(
    "null_group",
    F.sum(F.when(F.col("is_null_consumption") == 0, 1).otherwise(0)).over(window_spec_desc)
)

# Étape 6 : Calculer les durées des périodes nulles
data_grouped = data_filtered.groupBy("compteur", "null_group").agg(
    F.min("date").alias("start_date"),
    F.max("date").alias("end_date"),
    F.sum("is_null_consumption").alias("null_days")
)

# Conversion des dates en format `dd/MM/yyyy`
data_grouped = data_grouped.withColumn("start_date", F.date_format("start_date", "dd/MM/yyyy"))
data_grouped = data_grouped.withColumn("end_date", F.date_format("end_date", "dd/MM/yyyy"))

# Calculer la durée de chaque série nulle
data_grouped = data_grouped.withColumn(
    "duration",
    F.datediff(F.to_date("end_date", "dd/MM/yyyy"), F.to_date("start_date", "dd/MM/yyyy"))
).filter(F.col("null_days") > 0)

# Étape 7 : Calculer la durée moyenne des périodes nulles (v4)
dataV4 = data_grouped.groupBy("compteur").agg(
    F.avg("duration").alias("v4")
)

# Afficher le résultat final
dataV4.show(10)


+--------------------+------------------+
|            compteur|                v4|
+--------------------+------------------+
|C04AE134021_9,845...|2.4473684210526314|
|C07AA092853_9,894...| 5.895833333333333|
|C08EB001382_9,852...|11.514285714285714|
|C08FA041975_9,805...| 4.981481481481482|
|C08FA085851_9,849...|              4.55|
|C09FA030008_9,820...|               7.0|
|C09FA079673_9,881...|2.9473684210526314|
|C10ED005356_9,868...| 2.323529411764706|
|C10FA046664_9,851...|              16.0|
+--------------------+------------------+



#Variable v5 : Durée maximale des périodes de consommation nulles

In [156]:
# Étape 7 : Calculer la durée maximale des périodes nulles (v5)
datav5 = data_grouped.groupBy("compteur").agg(
    F.max("duration").alias("v5")
)

# Afficher les résultats finaux
datav5.show()

+--------------------+---+
|            compteur| v5|
+--------------------+---+
|C04AE134021_9,845...| 24|
|C07AA092853_9,894...| 48|
|C08EB001382_9,852...| 79|
|C08FA041975_9,805...|113|
|C08FA085851_9,849...| 16|
|C09FA030008_9,820...| 16|
|C09FA079673_9,881...| 25|
|C10ED005356_9,868...|  9|
|C10FA046664_9,851...| 89|
+--------------------+---+



# Variable v6 : longueur maximale de consommation non nulle

In [157]:
#reimportation des variable
data = spark.read.csv('/content/dataset_sample_spark.csv',
                      header=True,
                      inferSchema=True,
                      sep=';')  # Spécifie le point-virgule comme séparateur

In [158]:


# Initialiser SparkSession
spark = SparkSession.builder.appName("AnalyseConsommationNonNulle").getOrCreate()



# Étape 1 : Créer une colonne qui identifie chaque compteur
data = data.withColumn("compteur", F.concat_ws("_", F.col("sensor"), F.col("servicePoint"), F.col("transmitter")))

# Étape 2 : Identifier les séries de consommation non nulle
# Définir une fenêtre ordonnée par date pour chaque compteur
window_spec = Window.partitionBy("compteur").orderBy("date")

# Ajouter une colonne pour décaler les dates et identifier les séries strictement croissantes
data = data.withColumn("prev_date", F.lag("date").over(window_spec))
data = data.withColumn("prev_volume", F.lag("volume_l_value").over(window_spec))

# Identifier les débuts de séries non nulles
data = data.withColumn(
    "new_series",
    F.when(
        (F.col("volume_l_value") > 0) &  # La consommation actuelle est non nulle
        ((F.col("prev_volume") <= 0) | (F.col("prev_date").isNull()) |  # La précédente était nulle ou manquante
         (F.datediff(F.col("date"), F.col("prev_date")) > 1)),  # Ou l'indice précédent n'est pas consécutif
        1
    ).otherwise(0)
)

# Étape 3 : Assigner un identifiant unique à chaque série non nulle
data = data.withColumn("series_id", F.sum("new_series").over(window_spec))

# Étape 4 : Calculer la durée de chaque série non nulle
data_grouped = data.filter(F.col("volume_l_value") > 0).groupBy("compteur", "series_id").agg(
    F.min("date").alias("start_date"),
    F.max("date").alias("end_date"),
    (F.count("date") - 1).alias("number_of_days")  # Nombre de jours dans la série
)

# Étape 5 : Calculer la durée maximale des séries non nulles pour chaque compteur
dataV6 = data_grouped.groupBy("compteur").agg(
    F.max("number_of_days").alias("v6")
)

# Afficher les résultats finaux
dataV6.show()


+--------------------+---+
|            compteur| v6|
+--------------------+---+
|C04AE134021_9,845...| 30|
|C07AA092853_9,894...| 13|
|C08EB001382_9,852...|  8|
|C08FA041975_9,805...| 13|
|C08FA085851_9,849...| 18|
|C09FA030008_9,820...| 30|
|C09FA079673_9,881...|  8|
|C09FA127701_9,856...|866|
|C10ED005356_9,868...| 30|
|C10FA046664_9,851...| 16|
+--------------------+---+



# Variable v7 : Dernier indice de consommation

In [159]:
#reimportation des variable
data = spark.read.csv('/content/dataset_sample_spark.csv',
                      header=True,
                      inferSchema=True,
                      sep=';')  # Spécifie le point-virgule comme séparateur

In [160]:

# Initialiser SparkSession
spark = SparkSession.builder.appName("DernierIndiceConsommation").getOrCreate()


# Étape 1 : Créer une colonne qui identifie chaque compteur
data = data.withColumn("compteur", F.concat_ws("_", F.col("sensor"), F.col("servicePoint"), F.col("transmitter")))

# Étape 2 : Identifier la date To (dernière date significative) pour chaque compteur
window_spec = Window.partitionBy("compteur").orderBy(F.desc("date"))
data = data.withColumn("To", F.max("date").over(window_spec))

# Étape 3 : Filtrer les lignes correspondant à la date To
data_at_To = data.filter(F.col("date") == F.col("To"))

# Étape 4 : Extraire l'indice mécanique (volume_l_value) au To
data_v7 = data_at_To.select(
    "compteur",
    F.col("index_interpolated_d_l_value").alias("v7")  # Renommer pour clarité
)

# Afficher les résultats finaux
data_v7.show()


+--------------------+---------+
|            compteur|       v7|
+--------------------+---------+
|C04AE134021_9,845...|6713301.0|
|C07AA092853_9,894...|3526363.0|
|C08EB001382_9,852...| 543614.0|
|C08FA041975_9,805...| 600479.0|
|C08FA085851_9,849...|1207766.0|
|C09FA030008_9,820...|2132585.0|
|C09FA079673_9,881...|1694187.0|
|C09FA127701_9,856...|2808812.0|
|C10ED005356_9,868...|2006363.0|
|C10FA046664_9,851...| 426045.0|
+--------------------+---------+



# Variable v8 Nombre de périodes de consommation.

In [161]:
#reimportation des variable
data = spark.read.csv('/content/dataset_sample_spark.csv',
                      header=True,
                      inferSchema=True,
                      sep=';')  # Spécifie le point-virgule comme séparateur

In [162]:

# Initialiser SparkSession
spark = SparkSession.builder.appName("NombrePeriodesConsommation").getOrCreate()


# Étape 1 : Créer une colonne qui identifie chaque compteur
data = data.withColumn("compteur", F.concat_ws("_", F.col("sensor"), F.col("servicePoint"), F.col("transmitter")))

data = data.withColumn("date", F.to_date("date", "dd/MM/yyyy"))

# Étape 2 : Identifier la date To pour chaque compteur
window_spec = Window.partitionBy("compteur").orderBy(F.desc("date"))
data = data.withColumn("To", F.max("date").over(window_spec))

# Étape 3 : Filtrer les données sur une période d'un an avant To
data = data.withColumn(
    "within_one_year",
    F.when(F.datediff(F.col("To"), F.col("date")) <= 365, 1).otherwise(0)
)
data_one_year = data.filter(F.col("within_one_year") == 1)

# Étape 4 : Identifier les périodes de consommation strictement positive
window_spec = Window.partitionBy("compteur").orderBy("date")
data_one_year = data_one_year.withColumn("prev_volume", F.lag("volume_l_value").over(window_spec))

data_one_year = data_one_year.withColumn(
    "new_series",
    F.when(
        (F.col("volume_l_value") > 0) &  # Consommation positive actuelle
        ((F.col("prev_volume") <= 0) | F.col("prev_volume").isNull()),  # Début d'une nouvelle série
        1
    ).otherwise(0)
)

# Étape 5 : Compter le nombre de séries pour chaque compteur
data_one_year = data_one_year.withColumn("series_id", F.sum("new_series").over(window_spec))
datav8 = data_one_year.groupBy("compteur","To").agg(
    F.countDistinct("series_id").alias("V8")
)

# Afficher les résultats finaux
datav8.show()


+--------------------+----------+---+
|            compteur|        To| V8|
+--------------------+----------+---+
|C04AE134021_9,845...|2021-08-08|  7|
|C07AA092853_9,894...|2021-02-06| 30|
|C08EB001382_9,852...|2021-02-21| 21|
|C08FA041975_9,805...|2021-04-14| 32|
|C08FA085851_9,849...|2021-04-02|  9|
|C09FA030008_9,820...|2021-08-25|  7|
|C09FA079673_9,881...|2021-02-20| 77|
|C09FA127701_9,856...|2021-05-16|  1|
|C10ED005356_9,868...|2021-05-30| 10|
|C10FA046664_9,851...|2021-06-05|  6|
+--------------------+----------+---+



# Variable v9 : Dans le dataset training, le champ Millesime désigne les 5 premiers caractères du sensor.

### Nous commencons par faire quelque  transformation sur le variables et les dataframe intermediaire obtenu

In [163]:
from pyspark.sql import SparkSession

# Créer une Spark session
spark = SparkSession.builder \
    .appName("Fusion DataFrames") \
    .getOrCreate()

In [164]:
# Ajouter les colonnes de chaque DataFrame
result_df1 = dataV1 \
    .join(dataV2, on='compteur', how='left') \
    .join(dataV3, on='compteur', how='left') \
    .join(dataV4, on='compteur', how='left') \
    .join(datav5, on='compteur', how='left') \
    .join(dataV6, on='compteur', how='left') \
    .join(data_v7, on='compteur', how='left') \
    .join(datav8, on='compteur', how='left') \



# Afficher le résultat
result_df1.select("compteur", "v1" , "V2","V3","V3", "v4" , "v5", "v6",  "v7", "V8","To").show()

+--------------------+---+-------------------+-------------------+-------------------+------------------+----+---+---------+---+----------+
|            compteur| v1|                 V2|                 V3|                 V3|                v4|  v5| v6|       v7| V8|        To|
+--------------------+---+-------------------+-------------------+-------------------+------------------+----+---+---------+---+----------+
|C04AE134021_9,845...|366|0.07793212482141205|  0.198451011928361|  0.198451011928361|2.4473684210526314|  24| 30|6713301.0|  7|2021-08-08|
|C07AA092853_9,894...|366|0.20242521917372824| 0.2009644806973564| 0.2009644806973564| 5.895833333333333|  48| 13|3526363.0| 30|2021-02-06|
|C08EB001382_9,852...| 61|0.08178320241738442|0.23286648006513086|0.23286648006513086|11.514285714285714|  79|  8| 543614.0| 21|2021-02-21|
|C08FA041975_9,805...|366|0.12098275646939412|0.21015767843581087|0.21015767843581087| 4.981481481481482| 113| 13| 600479.0| 32|2021-04-14|
|C08FA085851_9,849..

In [165]:
# Afficher le résultat
result_df1.show(20)

+--------------------+----------+---+------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+----+---+---------+----------+---+
|            compteur|        T0| v1|       sum_91_days|         total_sum|                 V2|      sum_183_days|         total_sum|                 V3|                v4|  v5| v6|       v7|        To| V8|
+--------------------+----------+---+------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+----+---+---------+----------+---+
|C04AE134021_9,845...|2019-12-31|366|         113344.75|1454403.4345238106|0.07793212482141205|288627.83333333395|1454403.4345238106|  0.198451011928361|2.4473684210526314|  24| 30|6713301.0|2021-08-08|  7|
|C07AA092853_9,894...|2019-12-31|366| 119234.2333333334| 589028.5500000003|0.20242521917372824|118373.81666666688| 589028.5500000003| 0.2009644806973564| 5.895833333333333|

### Nous selectionnons  pour chaque compteur les differente variables

In [166]:
# Sélection des variables
selected_columns = ["compteur", "v1" , "V2","V3", "v4" , "v5", "v6",  "v7", "V8","T0"]
df_selected = result_df1.select(*[col for col in selected_columns if col in result_df1.columns])

df_selected.show()

+--------------------+---+-------------------+-------------------+------------------+----+---+---------+---+----------+
|            compteur| v1|                 V2|                 V3|                v4|  v5| v6|       v7| V8|        T0|
+--------------------+---+-------------------+-------------------+------------------+----+---+---------+---+----------+
|C04AE134021_9,845...|366|0.07793212482141205|  0.198451011928361|2.4473684210526314|  24| 30|6713301.0|  7|2019-12-31|
|C07AA092853_9,894...|366|0.20242521917372824| 0.2009644806973564| 5.895833333333333|  48| 13|3526363.0| 30|2019-12-31|
|C08EB001382_9,852...| 61|0.08178320241738442|0.23286648006513086|11.514285714285714|  79|  8| 543614.0| 21|2020-10-31|
|C08FA041975_9,805...|366|0.12098275646939412|0.21015767843581087| 4.981481481481482| 113| 13| 600479.0| 32|2019-12-31|
|C08FA085851_9,849...|  0|0.11176456142361486|0.22177504981590856|              4.55|  16| 18|1207766.0|  9|2020-12-31|
|C09FA030008_9,820...|  0|0.097538428434

In [167]:
#reimportation des variable
data = spark.read.csv('/content/dataset_sample_spark.csv',
                      header=True,
                      inferSchema=True,
                      sep=';')  # Spécifie le point-virgule comme séparateur

In [168]:
from pyspark.sql import functions as F

# Étape 1 : Créer une colonne qui identifie chaque compteur
data = data.withColumn("compteur", F.concat_ws("_", F.col("sensor"), F.col("servicePoint"), F.col("transmitter")))
# Ajouter une colonne pour extraire le millésime
data = data.withColumn("millesime", F.col("sensor").substr(1, 5))

window_spec = Window.partitionBy("compteur").orderBy("millesime")

data.show()

+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+--------------------+---------+
|     sensor|servicePoint|   client|transmitter|      date|meterDiameter|volume_l_value|index_interpolated_d_l_value|            compteur|millesime|
+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+--------------------+---------+
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|01/01/2019|           40|           0.0|                   5623306.0|C04AE134021_9,845...|    C04AE|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|02/01/2019|           40|          61.0|                   5623306.0|C04AE134021_9,845...|    C04AE|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|03/01/2019|           40|           7.0|                   5623367.0|C04AE134021_9,845...|    C04AE|
|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|04/01/2019|           40|           5.0|                  

In [169]:
df1 = df_selected  \
    .join(data, on='compteur', how='left')

In [170]:
df1.show()

+--------------------+---+-------------------+-----------------+------------------+---+---+---------+---+----------+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+---------+
|            compteur| v1|                 V2|               V3|                v4| v5| v6|       v7| V8|        T0|     sensor|servicePoint|   client|transmitter|      date|meterDiameter|volume_l_value|index_interpolated_d_l_value|millesime|
+--------------------+---+-------------------+-----------------+------------------+---+---+---------+---+----------+-----------+------------+---------+-----------+----------+-------------+--------------+----------------------------+---------+
|C04AE134021_9,845...|366|0.07793212482141205|0.198451011928361|2.4473684210526314| 24| 30|6713301.0|  7|2019-12-31|C04AE134021| 9,84588E+11|LYONNAISE|   C01E00D9|08/08/2021|           40|           0.0|                   7070442.0|    C04AE|
|C04AE134021_9,845...|366|0.

In [171]:
# Sélection des variables
selected_columns = ["compteur","client", "v1" , "V2","V3", "v4" , "v5", "v6",  "v7", "V8","millesime","T0"]
df = df1.select(*[col for col in selected_columns if col in df1.columns])

data=df.distinct()
data.show()

+--------------------+---------+---+-------------------+-------------------+------------------+----+---+---------+---+---------+----------+
|            compteur|   client| v1|                 V2|                 V3|                v4|  v5| v6|       v7| V8|millesime|        T0|
+--------------------+---------+---+-------------------+-------------------+------------------+----+---+---------+---+---------+----------+
|C04AE134021_9,845...|LYONNAISE|366|0.07793212482141205|  0.198451011928361|2.4473684210526314|  24| 30|6713301.0|  7|    C04AE|2019-12-31|
|C07AA092853_9,894...|LYONNAISE|366|0.20242521917372824| 0.2009644806973564| 5.895833333333333|  48| 13|3526363.0| 30|    C07AA|2019-12-31|
|C08EB001382_9,852...|LYONNAISE| 61|0.08178320241738442|0.23286648006513086|11.514285714285714|  79|  8| 543614.0| 21|    C08EB|2020-10-31|
|C08FA041975_9,805...|LYONNAISE|366|0.12098275646939412|0.21015767843581087| 4.981481481481482| 113| 13| 600479.0| 32|    C08FA|2019-12-31|
|C08FA085851_9,849..

### Ainsi dans ce dataFrame nous vvoyons pour chaque compteur les differentes variable tel que definis dans l'énnoncé

# nous allons maintenat creer la variable v9 selons les conditions definis

In [172]:
from pyspark.sql.functions import when, col, length

def calculate_v9(df):
    # Extraction des 5 premiers caractères de la colonne 'Millesime'
    df = df.withColumn("Millesime", col("Millesime").substr(1, 5))  # substr commence à 1 en PySpark

    # Conditions pour définir v9
    condition1 = (col("Millesime").isin(['C10FA', 'C10LA', 'C10SA', 'C11FA', 'C11LA', 'C11SA'])) & (length(col("V7")) > 3)
    condition2 = col("Millesime").isin(['D16BU', 'Z12ER', 'C07AA'])

    # Calcul de v9 selon les règles
    df = df.withColumn(
        "v9",
        when(condition1, 0).when(condition2, 1).otherwise(2)  # 2 par défaut
    )

    return df.distinct()



In [173]:
data_sample=calculate_v9(df=data)
data_sample.show()

+--------------------+---------+---+-------------------+-------------------+------------------+----+---+---------+---+---------+----------+---+
|            compteur|   client| v1|                 V2|                 V3|                v4|  v5| v6|       v7| V8|Millesime|        T0| v9|
+--------------------+---------+---+-------------------+-------------------+------------------+----+---+---------+---+---------+----------+---+
|C04AE134021_9,845...|LYONNAISE|366|0.07793212482141205|  0.198451011928361|2.4473684210526314|  24| 30|6713301.0|  7|    C04AE|2019-12-31|  2|
|C07AA092853_9,894...|LYONNAISE|366|0.20242521917372824| 0.2009644806973564| 5.895833333333333|  48| 13|3526363.0| 30|    C07AA|2019-12-31|  1|
|C08EB001382_9,852...|LYONNAISE| 61|0.08178320241738442|0.23286648006513086|11.514285714285714|  79|  8| 543614.0| 21|    C08EB|2020-10-31|  2|
|C08FA041975_9,805...|LYONNAISE|366|0.12098275646939412|0.21015767843581087| 4.981481481481482| 113| 13| 600479.0| 32|    C08FA|2019-12-

Nous obtenons ainsi le tableaau initiale (tran_data) de la partie une. a partir de ce tableau nous allons maintenant applique les calcule et les lancer des modele afin de pouvoir predire

#  Appliquons ainsi le modèle obtenu précédemment(Bagging)

# Implémentation d'un BaggingClassifier en PySpark

Nous allons appliqué le baggin pour predire si un compteur a le label STOPPED ou Blocked. nous avons deja entrainer le modele sur les données d'entrainement dans la premiere partir.

In [174]:

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql.functions import rand
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

### etape1: importation de notre modele

In [175]:
from pyspark.sql import SparkSession
import joblib

In [176]:
# Charger le modèle sauvegardé
loaded_model = joblib.load('bagging_model.pkl')

# Vérification : effectuer une prédiction avec le modèle chargé
y_pred = loaded_model.predict(X_test)




### etape2:  Préparer les nouvelles données dans Spark

In [177]:

# Sélectionner des colonnes spécifiques
data_train_spark = data_sample.select("compteur", "v1", "V2", "V3", "v4", "v5", "v6", "v7", "V8", "v9")

# Afficher le résultat
data_train_spark.show()

+--------------------+---+-------------------+-------------------+------------------+----+---+---------+---+---+
|            compteur| v1|                 V2|                 V3|                v4|  v5| v6|       v7| V8| v9|
+--------------------+---+-------------------+-------------------+------------------+----+---+---------+---+---+
|C04AE134021_9,845...|366|0.07793212482141205|  0.198451011928361|2.4473684210526314|  24| 30|6713301.0|  7|  2|
|C07AA092853_9,894...|366|0.20242521917372824| 0.2009644806973564| 5.895833333333333|  48| 13|3526363.0| 30|  1|
|C08EB001382_9,852...| 61|0.08178320241738442|0.23286648006513086|11.514285714285714|  79|  8| 543614.0| 21|  2|
|C08FA041975_9,805...|366|0.12098275646939412|0.21015767843581087| 4.981481481481482| 113| 13| 600479.0| 32|  2|
|C08FA085851_9,849...|  0|0.11176456142361486|0.22177504981590856|              4.55|  16| 18|1207766.0|  9|  2|
|C09FA030008_9,820...|  0|0.09753842843436737|0.18731148740529122|               7.0|  16| 30|21

### Utilisez un Pandas UDF pour appliquer le modèle Bagging aux données Spark

In [178]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pandas as pd

@pandas_udf("double", PandasUDFType.SCALAR)
def predict_udf(*cols) -> pd.Series:
    # Transformer les colonnes en une matrice d'entrée
    features = pd.concat(cols, axis=1).values
    # Faire des prédictions avec le modèle chargé
    predictions = bagging_model.predict(features)
    return pd.Series(predictions)


/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


## Appliquer la prédiction au DataFrame Spark

In [180]:
# Liste des colonnes d'entrée
feature_columns = ["V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9"]

# Ajouter une colonne de prédictions
df_with_predictions = data_train_spark.withColumn("prediction", predict_udf(*[data_train_spark[col] for col in feature_columns]))

# Afficher les résultats
df_with_predictions.show()


+--------------------+---+-------------------+-------------------+------------------+----+---+---------+---+---+----------+
|            compteur| v1|                 V2|                 V3|                v4|  v5| v6|       v7| V8| v9|prediction|
+--------------------+---+-------------------+-------------------+------------------+----+---+---------+---+---+----------+
|C04AE134021_9,845...|366|0.07793212482141205|  0.198451011928361|2.4473684210526314|  24| 30|6713301.0|  7|  2|       1.0|
|C07AA092853_9,894...|366|0.20242521917372824| 0.2009644806973564| 5.895833333333333|  48| 13|3526363.0| 30|  1|       1.0|
|C08EB001382_9,852...| 61|0.08178320241738442|0.23286648006513086|11.514285714285714|  79|  8| 543614.0| 21|  2|       1.0|
|C08FA041975_9,805...|366|0.12098275646939412|0.21015767843581087| 4.981481481481482| 113| 13| 600479.0| 32|  2|       1.0|
|C08FA085851_9,849...|  0|0.11176456142361486|0.22177504981590856|              4.55|  16| 18|1207766.0|  9|  2|       0.0|
|C09FA03